In [1]:
import pandas as pd
import numpy as np

data17 = pd.read_csv("properati-AR-2017-08-01-properties-sell.csv", low_memory = False)
data16 = pd.read_csv("properati-AR-2016-12-01-properties-sell.csv", low_memory = False)
data15 = pd.read_csv("properati-AR-2015-12-01-properties-sell.csv", low_memory = False)
data14 = pd.read_csv("properati-AR-2015-12-01-properties-sell.csv", low_memory = False)
frames = [data14, data15, data16, data17]
data = pd.concat(frames)
data.head()

,country_name,created_on,currency,description,expenses,extra,floor,geonames_id,id,image_thumbnail,...,price_aprox_usd,price_per_m2,price_usd_per_m2,properati_url,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,title
0,NaN,2012-08-08,NaN,Excelente depto. en planta alta con patio y co...,NaN,"{""images"":[""http://www.amaira.com.ar/buscador/...",NaN,3433778.0,NaN,http://thumbs3.properati.com/2/hrxzmGbOVe0AAWf...,...,NaN,NaN,NaN,http://general-san-martin-bs-as-g-b-a-zona-nor...,apartment,3.0,NaN,NaN,NaN,DEPARTAMENTO EN VENTA
1,NaN,2012-08-08,NaN,Venta chalet 4 ambientes en Villa Ballester e...,NaN,"{""images"":[""http://www.amaira.com.ar/buscador/...",NaN,3427467.0,NaN,http://thumbs3.properati.com/5/a4W8KzDBTc_SzTL...,...,NaN,NaN,NaN,http://villa-ballester.properati.com.ar/be_ven...,house,5.0,NaN,480.0,480.0,CASA EN VENTA
2,NaN,2012-09-19,USD,CASA CON LOCAL EN VENTA EN PADUA. 2 DORMITORI...,NaN,"{""images"":[""http://www.amaira.com.ar/buscador/...",NaN,3429244.0,NaN,http://thumbs3.properati.com/3/4WSbwKA0arL7bbQ...,...,120000.0,1290.322581,360.36036,http://san-antonio-de-padua.properati.com.ar/u...,house,NaN,NaN,93.0,333.0,CASA EN VENTA
3,NaN,2012-10-10,USD,"2 AMBIENTES, VENTA, VILLA CRESPO CON RENTA....",50,"{""apto_profesional"":true,""images"":[""http://www...",1.0,3427458.0,NaN,http://thumbs3.properati.com/4/hqn-p5RqpJu7eVj...,...,76500.0,1912.500000,1912.50000,http://villa-crespo.properati.com.ar/13tz_vent...,apartment,2.0,NaN,40.0,40.0,DEPARTAMENTO EN VENTA
4,NaN,2012-10-10,USD,"CASA, VENTA, VILLA CRESPO ""EXCELENTE CASA E...",NaN,"{""baulera"":true,""dependencia"":true,""apto_profe...",NaN,3430234.0,NaN,http://thumbs3.properati.com/6/szVEIqerNvrkp-o...,...,570000.0,1727.272727,1425.00000,http://palermo.properati.com.ar/13v3_venta_dep...,apartment,5.0,NaN,330.0,400.0,DEPARTAMENTO EN VENTA


In [2]:
#Solo me quedo con CAP FED y GBA
dataCapGba = data.loc[(data.state_name.str.contains('Capital Federal') | data.state_name.str.contains('G.B.A') ), ['created_on', 'property_type', 'lat', 'lon', 'price', 'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'rooms']]
dataCapGba = dataCapGba.loc[dataCapGba['price_usd_per_m2'] < 9000,:]
dataCapGba = dataCapGba.loc[dataCapGba['price_usd_per_m2'] > 1000,:]
dataCapGba = dataCapGba.loc[dataCapGba['surface_total_in_m2'] > 5, :]

In [3]:
#Elimino duplicados y registros con ALGUNA COLUMNA NaN (no nos serviran para predecir)
dataCapGba.drop_duplicates(subset=['lat', 'lon', 'price_usd_per_m2'])
dataCapGba.dropna(inplace=True, how='any')
dataCapGba.head()

,created_on,property_type,lat,lon,price,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms
7,2016-06-01,house,-34.439418,-58.925763,215000.0,138.0,108.0,1557.971014,1990.740741,4.0
8,2016-06-01,house,-34.402935,-58.826622,175000.0,165.0,165.0,1060.606061,1060.606061,4.0
9,2016-06-01,house,-34.437969,-58.988003,250000.0,240.0,240.0,1041.666667,1041.666667,4.0
11,2016-06-01,house,-34.479254,-58.919930,540000.0,440.0,360.0,1227.272727,1500.000000,9.0
12,2016-06-01,house,-34.486971,-58.919963,237000.0,202.0,150.0,1173.267327,1580.000000,5.0


In [4]:
# Importando librerias necesarias para ML
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [5]:
#Transformemos las columnas created_on y property_type a valores numericos para poder usarlos a la hora de predecir
#Para ello, crearemos algunas funciones diccionario que nos sirvan para dicha informacion:

#property_type's: apartament, house, PH, store
def tipoDePropiedad(x):
    tipos = {
        "apartment" : 1,
        "house" : 2,
        "PH" : 3,
        "store" : 4
    }
    return tipos[x] if x in tipos else x

from datetime import datetime
#created_on's: yyyy-mm-dd
def fechaNumerica(x):
    d = datetime.strptime('2012-02-10' , '%Y-%m-%d')
    return int("%4d%02d%02d" % (d.year, d.month, d.day)) + 0

In [6]:
#Aplicamos las funciones anteriormente creadas
dataCapGba.loc[:, 'property_type'] = dataCapGba.property_type.apply(tipoDePropiedad)
dataCapGba.loc[:, 'created_on'] = dataCapGba.created_on.apply(fechaNumerica)

In [7]:
#Ahora, todas las columnas son numericas
dataCapGba.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32828 entries, 7 to 186306
Data columns (total 10 columns):
created_on               32828 non-null int64
property_type            32828 non-null int64
lat                      32828 non-null float64
lon                      32828 non-null float64
price                    32828 non-null float64
surface_total_in_m2      32828 non-null float64
surface_covered_in_m2    32828 non-null float64
price_usd_per_m2         32828 non-null float64
price_per_m2             32828 non-null float64
rooms                    32828 non-null float64
dtypes: float64(8), int64(2)
memory usage: 2.8 MB


In [8]:
#Convertimos todo a int
testData = dataCapGba
testData.loc[:, 'lat'] = testData.lat.apply(lambda x: int(x))
testData.loc[:, 'lon'] = testData.lon.apply(lambda x: int(x))
testData.loc[:, 'price'] = testData.price.apply(lambda x: int(x))
testData.loc[:, 'surface_total_in_m2'] = testData.surface_total_in_m2.apply(lambda x: int(x))
testData.loc[:, 'surface_covered_in_m2'] = testData.surface_covered_in_m2.apply(lambda x: int(x))
testData.loc[:, 'price_usd_per_m2'] = testData.price_usd_per_m2.apply(lambda x: int(x))
testData.loc[:, 'price_per_m2'] = testData.price_per_m2 .apply(lambda x: int(x))
testData.loc[:, 'rooms'] = testData.rooms.apply(lambda x: int(x))
SEED = 7

array = testData.values
X = array[:, :]
#Quiero estimar 'price_usd_per_m2'
Y = array[:, 7]
datosValidacion = 0.20
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=datosValidacion, random_state=SEED)

In [9]:
testData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32828 entries, 7 to 186306
Data columns (total 10 columns):
created_on               32828 non-null int64
property_type            32828 non-null int64
lat                      32828 non-null int64
lon                      32828 non-null int64
price                    32828 non-null int64
surface_total_in_m2      32828 non-null int64
surface_covered_in_m2    32828 non-null int64
price_usd_per_m2         32828 non-null int64
price_per_m2             32828 non-null int64
rooms                    32828 non-null int64
dtypes: int64(10)
memory usage: 2.8 MB


In [ ]:
#Evaluando distintos algoritmos..
SCORING = 'accuracy'

aModelos = []
aModelos.append(('LR', LogisticRegression()))
aModelos.append(('LDA', LinearDiscriminantAnalysis()))
aModelos.append(('KNN', KNeighborsClassifier()))
aModelos.append(('CART', DecisionTreeClassifier()))
aModelos.append(('NB', GaussianNB()))
aModelos.append(('SVM', SVC()))
aResultados = []
aNombres = []
for nombre, modelo in aModelos:
    kfold = model_selection.KFold(n_splits=10, random_state=SEED)
    res = model_selection.cross_val_score(modelo, X_train, Y_train, cv=kfold, scoring=SCORING)
    aResultados.append(res)
    aNombres.append(nombre)
    print "%s: %f (%f)" % (nombre, res.mean(), res.std())